In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
input_size = 28 * 28  # start cells.
hidden_size = 128  # middle cells.
num_classes = 10  # answer cells.
num_epochs = 10  # how many times it goes through data.
batch_size = 100  # how many data I give it every time.
learning_rate = 0.001  # how big its steps will be.

In [7]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
# print(transform)
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



# with one hidden layer

In [8]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


model = MLP(input_size, hidden_size, num_classes)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28)
        labels = labels



        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/10], Step [100/600], Loss: 0.4036
Epoch [1/10], Step [200/600], Loss: 0.5317
Epoch [1/10], Step [300/600], Loss: 0.2899
Epoch [1/10], Step [400/600], Loss: 0.1805
Epoch [1/10], Step [500/600], Loss: 0.1753
Epoch [1/10], Step [600/600], Loss: 0.1475
Epoch [2/10], Step [100/600], Loss: 0.1545
Epoch [2/10], Step [200/600], Loss: 0.1956
Epoch [2/10], Step [300/600], Loss: 0.1970
Epoch [2/10], Step [400/600], Loss: 0.0617
Epoch [2/10], Step [500/600], Loss: 0.0872
Epoch [2/10], Step [600/600], Loss: 0.0681
Epoch [3/10], Step [100/600], Loss: 0.0496
Epoch [3/10], Step [200/600], Loss: 0.0858
Epoch [3/10], Step [300/600], Loss: 0.2329
Epoch [3/10], Step [400/600], Loss: 0.2138
Epoch [3/10], Step [500/600], Loss: 0.1555
Epoch [3/10], Step [600/600], Loss: 0.0833
Epoch [4/10], Step [100/600], Loss: 0.0226
Epoch [4/10], Step [200/600], Loss: 0.1241
Epoch [4/10], Step [300/600], Loss: 0.0169
Epoch [4/10], Step [400/600], Loss: 0.0193
Epoch [4/10], Step [500/600], Loss: 0.0213
Epoch [4/10

In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the MLP on the test images: {} %'.format(100 * correct / total))

Accuracy of the MLP on the test images: 97.8 %
